In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# https://xungejiang.com/2020/06/06/BERT/
# https://juejin.cn/post/6844904167257931783#heading-21
# https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
# https://pytorch.org/docs/stable/search.html?q=schedule&check_keywords=yes&area=default#
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=c5aca1f468f2b67b41619d9c2c5183033371075e86c7c5a66db43297d2e86575
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
import wget
import os
import pandas as pd
from tqdm import tqdm

import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


print('Downloading dataset...')
url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

if not os.path.exists('./cola_public_1.1.zip'):
    wget.download(url, './cola_public_1.1.zip')

if not os.path.exists('./cola_public/'):
    !unzip cola_public_1.1.zip
    
df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

print('Number of training sentences: {:,}\n'.format(df.shape[0]))
df.sample(10)

Archive:  cola_public_1.1.zip
   creating: cola_public/
  inflating: cola_public/README      
   creating: cola_public/tokenized/
  inflating: cola_public/tokenized/in_domain_dev.tsv  
  inflating: cola_public/tokenized/in_domain_train.tsv  
  inflating: cola_public/tokenized/out_of_domain_dev.tsv  
   creating: cola_public/raw/
  inflating: cola_public/raw/in_domain_dev.tsv  
  inflating: cola_public/raw/in_domain_train.tsv  
  inflating: cola_public/raw/out_of_domain_dev.tsv  
Number of training sentences: 8,551



,sentence_source,label,label_notes,sentence
3990,ks08,0,*,They want to wrote to her.
3547,ks08,0,*,They read the under book.
3487,ks08,1,NaN,The room contains few armchairs.
4046,ks08,1,NaN,John intends to review the book.
5838,c_13,0,*,I've never seen you ate asparagus.
771,bc01,0,*,We love their.
2003,rhl07,1,NaN,Jill kicked the ball from home plate to third ...
5417,b_73,1,NaN,Harry got to be as much of a celebrity as his ...
7533,sks13,0,*,Mary noticed John's excessive appreciation of ...
7248,sks13,1,NaN,John asked whether she left.


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

sentences = df['sentence'].values
labels = df['label'].values

input_ids = []
attention_masks = []

for sentence in sentences:
    encode_dic = tokenizer.encode_plus(
        sentence,
        max_length= 64,
        add_special_tokens=True,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids.append(encode_dic['input_ids'])
    attention_masks.append(encode_dic['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

data = TensorDataset(input_ids, attention_masks, labels)
print(len(data))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


8551


In [6]:
train_size = int(0.9* len(data))
val_size = len(data) - train_size

train, val = random_split(data, [train_size, val_size])

train_Loader = DataLoader(train, batch_size=32, sampler=RandomSampler(train))
val_Loader = DataLoader(val, batch_size=32, sampler=SequentialSampler(val))

In [7]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', num_labels=2, output_attentions=False, output_hidden_states=False)
model.cuda()

optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
EPOCHS=4
average_loss = 0
for epoch in range(EPOCHS):
    
    model.train()
    total_train_loss = 0
    # train
    for step, batch in enumerate(train_Loader):
        
        batch_input_ids = batch[0].cuda()
        batch_attention_mask = batch[1].cuda()
        batch_labels = batch[2].cuda()
        
        model.zero_grad()
        
        loss, logit = model(batch_input_ids, 
                          token_type_ids=None,
                          attention_mask=batch_attention_mask, 
                          labels=batch_labels)
        
        loss.backward
        optimizer.step()
        
        total_train_loss += loss
        
    average_loss += total_train_loss / len(train_Loader)
        
        
    
        
        
        
        
        

KeyboardInterrupt: 